In [1]:
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pinecone import Pinecone, ServerlessSpec  
from langchain_pinecone import PineconeVectorStore

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

c:\Users\ASUS\anaconda3\envs\medchatEnv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')
index_name = os.environ.get('index_name')

#### Extract data from the PDF :

In [49]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [50]:
extracted_data = load_pdf("./data/")

In [7]:
extracted_data[0:5]

[Document(page_content='', metadata={'source': 'data\\Medical_book.pdf', 'page': 0}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': 'data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly 

In [8]:
len(extracted_data)

637

#### Create text chunks :

In [51]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [52]:
text_chunks = text_split(extracted_data)

In [11]:
len(text_chunks)

7020

In [12]:
text_chunks[0:5]

[Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': 'data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and', metadata={'source': 'data\\Medical_book.pdf', 'page': 3}),
 Document(page_content='Multimedia Content\nKelly A.

#### Download Embeddings :

In [3]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [4]:
embeddings = download_hugging_face_embeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [17]:
query_result

[-0.03447723761200905,
 0.031023172661662102,
 0.006734967231750488,
 0.02610894665122032,
 -0.03936203941702843,
 -0.1603025197982788,
 0.06692396849393845,
 -0.006441415287554264,
 -0.0474504753947258,
 0.014758882112801075,
 0.07087530195713043,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625138871371746,
 -0.010109545662999153,
 -0.026940535753965378,
 0.02230738289654255,
 -0.022226667031645775,
 -0.14969263970851898,
 -0.01749306730926037,
 0.007676335051655769,
 0.05435227230191231,
 0.003254442475736141,
 0.03172587975859642,
 -0.08462145179510117,
 -0.02940605953335762,
 0.05159563571214676,
 0.04812406376004219,
 -0.0033148345537483692,
 -0.058279212564229965,
 0.04196925461292267,
 0.022210633382201195,
 0.1281888782978058,
 -0.02233893983066082,
 -0.011656169779598713,
 0.06292835623025894,
 -0.03287630155682564,
 -0.09122607856988907,
 -0.031175361946225166,
 0.05269959196448326,
 0.04703481122851372,
 -0.0842030867934227,
 -0.030056176707148552,
 -0.020744783803820

#### Embedding chunks and storing them into pinecone :

In [7]:
pc = Pinecone(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)  

In [56]:
if index_name not in pc.list_indexes().names():
    print(index_name , " Does not exist !") 
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

In [59]:
index = pc.Index(index_name)  
index.describe_index_stats() 
# index.delete(delete_all=True, namespace='medicalChatBot')

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [60]:
vectorstore = PineconeVectorStore(
    index=index,
    pinecone_api_key = PINECONE_API_KEY,
    embedding=embeddings,
    # namespace="medicalChatBot",
    index_name=index_name
)

In [61]:
vectorstore.add_texts(texts=[t.page_content for t in text_chunks])

['eb7262dc-01cf-43cf-9432-e6074c1f0c7a',
 '8aaa1152-3aeb-422f-8dce-4a2c60f3558d',
 '300c8a50-34a9-4fa3-8f5c-6dfe514a6c84',
 '2b7dcd44-88fe-406d-b837-3a7155c7e9d1',
 'ea889573-5d12-4598-a784-a4273c0921ba',
 '77eabc8e-2637-411f-8acf-3f15ebd8dc4c',
 'bb8b0571-9efe-4d67-87fe-584e4636c3fa',
 '5ef37973-9fb0-42c4-bc32-fd85061479b8',
 'd778fe0f-b385-49f3-a0a0-e5a324698d79',
 'dc1ed084-d8c2-4b17-8e29-fff95a3450fc',
 '0c1ae2f9-2d9b-4b5d-bc8d-775d5cc32301',
 'b91dc2b3-a918-41c4-9fc0-22519d073b36',
 '2f26bf4b-4d93-457c-9feb-5ba9cfafe382',
 '0946d6f4-0c2d-4cee-9717-d3ccfce32efe',
 '74c1eced-0a59-462f-962f-7da2873d07da',
 '9099892d-abe2-43c2-8bc6-83a2ec6fe3a0',
 'c4bd4bbe-1165-44c9-b321-b28c78e72f6b',
 'fb1dcf6c-cda2-4106-b06b-bbff5a700b87',
 '5227bb7a-e8ad-4d05-91b1-ef45d1e7d027',
 '674179b5-fcea-41d6-a81a-f88cc64bc542',
 '3fbedeb0-01a1-4701-922f-f21403a393a2',
 '82b8a0b1-7c62-4264-aa55-13d0a764b133',
 '4f37b7f6-aa24-4c13-be4e-85dca6c55234',
 '5860240e-4154-4b84-ab3f-8d88fee64d13',
 '57d649fe-909f-

In [62]:
index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0702,
 'namespaces': {'': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [ ]:
# pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
#     # Creating Embeddings for Each of The Text Chunks & storing :
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)  
index = pc.Index(index_name)  

In [6]:
queryvectorstore = PineconeVectorStore(
    index=index,
    pinecone_api_key = PINECONE_API_KEY,
    embedding=embeddings,
    index_name=index_name
)

In [7]:
query = "What are Allergies"
docs = queryvectorstore.similarity_search(query, k=5)
print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [ ]:
#     #If we already have an index we can load it like this :
# docsearch=Pinecone.from_existing_index(index_name, embeddings)
# query = "What are Allergies"
# docs=docsearch.similarity_search(query, k=3)
# print("Result", docs)  # return ranked result which should be passed to the llama model 

#### Using the llama model:

In [8]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [9]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [10]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                model_type="llama",
                config={'max_new_tokens':512,
                        'temperature':0.8})

In [11]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=queryvectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


KeyboardInterrupt: 